This work is licensed under the Creative Commons Attribution 3.0 Unported License. To view a copy of this license, visit http://creativecommons.org/licenses/by/3.0/.

#### Importing libraries

In [ ]:
import json
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

#### Loading scrapped data in dataframe format

In [ ]:
with open('candidates-search/data/job_candidates_data.json') as f:
    use_case_1 = json.load(f)
    
with open('medical-specialists-search/data/medical_specialists_data.json') as f:
    use_case_2 = json.load(f)
    
use_case1_source = [candidate['_source'] for candidate in use_case_1['hits']['hits']]
use_case2_source = [candidate['_source'] for candidate in use_case_2['hits']['hits']]

df_u1 = pd.io.json.json_normalize(use_case1_source)
df_u2 = pd.io.json.json_normalize(use_case2_source)

#### Data gathered from Linkedin and Viadeo sites in Spain, France and United Kingdom

In [ ]:
queries_fr_l = df_u1[(df_u1['country'] == "France") & (df_u1['source'] == 'linkedin')]['query'].drop_duplicates()
queries_fr_v = df_u1[(df_u1['country'] == "France") & (df_u1['source'] == 'viadeo')]['query'].drop_duplicates()

queries_es_l = df_u1[(df_u1['country'] == "Spain") & (df_u1['source'] == 'linkedin')]['query'].drop_duplicates()
queries_es_v = df_u1[(df_u1['country'] == "Spain") & (df_u1['source'] == 'viadeo')]['query'].drop_duplicates()

queries_uk_l = df_u1[(df_u1['country'] == "United Kingdom") & (df_u1['source'] == 'viadeo')]['query'].drop_duplicates()
queries_uk_v = df_u1[(df_u1['country'] == "United Kingdom") & (df_u1['source'] == 'viadeo')]['query'].drop_duplicates()

#### Obtain gender proportion

In [ ]:
# Calculates gender proportion for the Top 10 and Top 20 positions of the ranking
def get_metrics_gender(df, source, country, queries, tag_queries='query'):
    print("Country: " + country + " Source: " + source)
    print("{: >20} {: >10} {: >10} {: >10} {: >10}{: >10} {: >10}".format("Q","%M#10", "%F#10", "%U#10","%M#20", "%F#20", "%U#20"))

    for q in sorted(queries):
        df_10 = df[(df[tag_queries] == q) & (df['source'] == source) & (df['country'] == country)].sort_values('ranking')[:10]
        df_20 = df[(df[tag_queries] == q) & (df['source'] == source) & (df['country'] == country)].sort_values('ranking')[:20]
        
        m_10 = round((len(df_10[(df_10['gender'] == 'male')]) / float(10)) * 100,3)
        f_10 = round((len(df_10[(df_10['gender'] == 'female')]) / float(10)) * 100,3)
        u_10 = round((len(df_10[(df_10['gender'] == 'unknown')]) / float(10)) * 100,3)
        
        m_20 = round((len(df_20[(df_20['gender'] == 'male')]) / float(20)) * 100,3)
        f_20 = round((len(df_20[(df_20['gender'] == 'female')]) / float(20)) * 100,3)
        u_20 = round((len(df_20[(df_20['gender'] == 'unknown')]) / float(20)) * 100,3)
        
        # If the query has assigned less than 10 people, the sum of proportions will not result in 100%
        print("{: >20} {: >10} {: >10} {: >10} {: >10}{: >10} {: >10}".format(q,m_10,f_10,u_10,m_20,f_20,u_20))


# Function to obtain queries that have at least a minimum number of people
def get_filtered_queries(df, queries, source, country, tag_queries='query', min_all_len = 20):
    filtered_queries = []

    #print("Country:" + country +" Source: "+ source)
    #print("{: >15} {: >15} {: >15} {: >15} {: >15}".format("Query", "All", "Male", "Female", "Other"))
    for query in sorted(queries):
        all_len = len(df[(df[tag_queries] == query) & (df['source'] == source) & (df['country'] == country)])
        female_len = len(df[(df[tag_queries] == query) & (df['source'] == source) & (df['gender']== 'female') & (df['country'] == country)])
        male_len = len(df[(df[tag_queries] == query) & (df['source'] == source) & (df['gender']== 'male') & (df['country'] == country)])
        other_len = all_len - (male_len + female_len)
        if all_len >= min_all_len:
            filtered_queries.append(query)
            #print("{: >15} {: >15} {: >15} {: >15} {: >15}".format(query,all_len,male_len,female_len,other_len))
    return filtered_queries

def get_filtered_queries_linkedin(country, queries):
    return get_filtered_queries(df_u1, queries, 'linkedin', country, min_all_len = 20)

def get_filtered_queries_viadeo(country, queries):
    return get_filtered_queries(df_u1, queries, 'viadeo', country,min_all_len = 20)            

#### Obtain proportion metrics per country and data source

In [ ]:
get_metrics_gender(df_u1, 'linkedin', 'Spain', queries_es_l)
get_metrics_gender(df_u1, 'viadeo', 'Spain', queries_es_v)
print("------------------------------------------------------------------------------------")
get_metrics_gender(df_u1, 'linkedin', 'United Kingdom', queries_uk_l)
get_metrics_gender(df_u1, 'viadeo', 'United Kingdom', queries_uk_v)
print("------------------------------------------------------------------------------------")
get_metrics_gender(df_u1, 'linkedin', 'France', queries_fr_l)
get_metrics_gender(df_u1, 'viadeo', 'France', queries_fr_v)

#### Calculate unknown rate per country

In [ ]:
unknown = len(df_u1[(df_u1['country']=='Spain') & (df_u1['gender']=='unknown')])
total = len(df_u1[(df_u1['country']=='Spain')])
print('ES rate unknown: ',round(unknown/float(total),2))

unknown = len(df_u1[(df_u1['country']=='United Kingdom') & (df_u1['gender']=='unknown')])
total = len(df_u1[(df_u1['country']=='United Kingdom')])
print('UK rate unknown: ',round(unknown/float(total),2))

unknown = len(df_u1[(df_u1['country']=='France') & (df_u1['gender']=='unknown')])
total = len(df_u1[(df_u1['country']=='France')])
print('FR rate unknown: ',round(unknown/float(total),2))

#### Data gathered from Top doctors sites in Spain, Colombia and Mexico

In [ ]:
queries_co = df_u2[(df_u2['country'] == "Colombia") & (df_u2['source'] == 'top doctors')]['speciality'].drop_duplicates()
queries_es = df_u2[(df_u2['country'] == "Spain") & (df_u2['source'] == 'top doctors')]['speciality'].drop_duplicates()
queries_mx = df_u2[(df_u2['country'] == "Mexico") & (df_u2['source'] == 'top doctors')]['speciality'].drop_duplicates()

In [ ]:
# Function to obtain specialities that have at least a minimum number of people
def get_filtered_specialisties(df, specialities, country, source = 'top doctors', min_all_len = 20):
    filtered_specialities = []

    #print("Country:" + country +" Source: "+ source)
    #print("{: >15} {: >15} {: >15} {: >15} {: >15}".format("Speciality", "All", "Male", "Female", "Other"))
    for speciality in sorted(specialities):
        all_len = len(df[(df['speciality'] == speciality) & (df['source'] == source) & (df['country'] == country)])
        female_len = len(df[(df['speciality'] == speciality) & (df['source'] == source) & (df['gender']== 'female') & (df['country'] == country)])
        male_len = len(df[(df['speciality'] == speciality) & (df['source'] == source) & (df['gender']== 'male') & (df['country'] == country)])
        other_len = all_len - (male_len + female_len)
        if all_len >= min_all_len:
            filtered_specialities.append(speciality)
            #print("{: >15} {: >15} {: >15} {: >15} {: >15}".format(speciality,all_len,male_len,female_len,other_len))
    return filtered_specialities

#### Getting subspecialities that the 3 countries have in common (intersection) with a minimum presence of 20 people

In [ ]:
f_queries_co = get_filtered_specialisties(df_u2, queries_co, 'Colombia')
f_queries_mx = get_filtered_specialisties(df_u2, queries_mx, 'Mexico')
f_queries_es = get_filtered_specialisties(df_u2, queries_es, 'Spain')

intersection = list(set(f_queries_co) & set(f_queries_mx) & set(f_queries_es))

print(intersection)

In [ ]:
get_metrics_gender(df_u2, 'top doctors', 'Colombia', intersection, 'speciality')
print("------------------------------------------------------------------------------------")
get_metrics_gender(df_u2, 'top doctors', 'Spain', intersection, 'speciality')
print("------------------------------------------------------------------------------------")
get_metrics_gender(df_u2, 'top doctors', 'Mexico', intersection, 'speciality')

#### Unknown rate per source

In [ ]:
medical_specialists_to_analyze = []
for speciality in intersection:
    medical_specialists_to_analyze.append(df_u2[(df_u2['speciality'] == speciality)])

df_u2_to_analyze = pd.concat(medical_specialists_to_analyze)
unknown = len(df_u2_to_analyze[df_u2_to_analyze['gender']=='unknown'])
total = len(df_u2_to_analyze)
print('rate unknown: ',round(unknown/float(total),3))

#### Unknown justification

In [ ]:
# Extracting a random sample of size 20 to see the names and surnames of the profiles gathered 
# to check why their gender is marked as unknown
def store_unknown_values(df, filename):
    f = open(filename + ".csv","w+")
    df_unknown = df[df['gender']=='unknown']
    #print("len unknown: ",len(df_unknown))
    f.write("len unknown: " + str(len(df_unknown)) + "\n\n")
    f.write(str(df_unknown)+"\n\n")


    # Get 20 random numbers (no repetition)
    import random
    random_ints=[]
    while(len(random_ints)<20):
        r = random.randint(1,len(df_unknown))
        if r not in random_ints:
            random_ints.append(r)
    #print(random_ints)
    f.write(str(random_ints)+"\n\n")

    # Get rows by index
    source_id = []
    for candidate in use_case_1['hits']['hits']:
        candidate['_source']['_id'] = candidate['_id']
        source_id.append(candidate['_source'])

    # Check full name
    df_id = pd.io.json.json_normalize(source_id)
    df_unknown = df_id[df_id['gender'] == 'unknown'][['_id','name','surnames','gender']]
    df_unknown.to_csv(filename + ".csv", sep='\t', encoding='utf-8')

store_unknown_values(df_u1, filename = 'unknown_job_candidates')
store_unknown_values(df_u2, filename = 'unknown_medical_specialists')

#### Storing variables to be able to use them in a different notebook

In [ ]:
%store df_u1
%store queries_fr_l
%store queries_fr_v
%store queries_es_l
%store queries_es_v
%store queries_uk_l
%store queries_uk_v
%store df_u2
%store intersection
%store f_queries_co
%store f_queries_mx
%store f_queries_es
%store intersection